> ℹ️ Notebook adapted from https://guidance.readthedocs.io

`The Art of Prompt Design`


# Prompt Boundaries and Token Healing

This (originally written by Scott Lundberg and <a href="https://medium.com/@marcotcr">Marco Tulio Ribeiro</a>) is part of a series about controlling large language models (LLMs) with <a href="https://github.com/microsoft/guidance">`guidance`</a>.

In this post, we'll discuss how the greedy tokenization methods used by language models can introduce unintended token splits into your prompts, leading to puzzling generations.

Language models are not trained on raw text, but rather on tokens, which are chunks of text that often occur together, similar to words. This impacts how language models 'see' text, including prompts (since prompts are just sets of tokens). GPT-style models utilize tokenization methods like [Byte Pair Encoding](https://en.wikipedia.org/wiki/Byte_pair_encoding) (BPE), which map all input bytes to token ids in an optimized/greedy manner. This is fine for training, but it can lead to subtle issues during inference, as shown in the example below.

<!-- TODO
Standard greedy token mapping works well during training, but it can lead to subtle issues during prompting and inference. These issues arise because the greedy token boundaries often don't line up with the end of the prompt, especially when considering the generated tokens that will come next. While the end of a prompt will always align with a token boundary in practice, as the prompt is tokenized before being extended by the model, there may be instances where the first characters of the completion are part of a longer token that would span the prompt boundary. In such cases, the longer token cannot be used even though the model would expect it based on the training data.

The inability to use tokens that span prompt boundaries can lead to subtle yet important biases in the model's output. -->


## An example of a prompt boundary problem
Consider the following example, where we are trying to generate an HTTP URL string:

In [16]:
import transformers
model = "gpt2"

# we use a specific as an example, but these issues impact all models to varying degrees
generator = transformers.pipeline('text-generation', model=model)

def raw_gen(prompt, temp=0):
    kwargs = {}
    if temp > 0:
        kwargs["temperature"] = temp
        kwargs["do_sample"] = True
    return generator(prompt, max_new_tokens=10, pad_token_id=0, **kwargs)[0]["generated_text"]
raw_gen('The link is <a href="http:')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


'The link is <a href="http:">http://www.reddit.com/r'

Note that the output generated by the LLM does not complete the url with the obvious next characters (two forward slashes). It instead creates an invalid URL string with a space in the middle. This is surprising, because the `//` completion is extremely obvious after `http:`. To understand why this happens, let's change our prompt boundary so that our prompt does not include the colon character:

In [17]:
raw_gen('The link is <a href="http')

'The link is <a href="http://twitter.com/#!/TroyTrot'

Now the language model generates a valid url string like we expect. To understand why the  `:` matters, we need to look at the tokenized representation of the prompts. Below is the tokenization of the prompt that ends in a colon (the prompt without the colon has the same tokenization, except for the last token):

In [18]:
def print_tokens(tokens):
    print("len = " + str(len(tokens)))
    for i in tokens:
        print(str(i) + "\t`" + generator.tokenizer.decode([i]) + "`")

print_tokens(generator.tokenizer.encode('The link is <a href="http:'))

len = 9
464	`The`
2792	` link`
318	` is`
1279	` <`
64	`a`
13291	` href`
2625	`="`
4023	`http`
25	`:`


Now note what the tokenization of a valid URL looks like, paying careful attention to token `1358`, right after `http`:

In [19]:
print_tokens(generator.tokenizer.encode('The link is <a href="http://www.google.com/search?q'))

len = 18
464	`The`
2792	` link`
318	` is`
1279	` <`
64	`a`
13291	` href`
2625	`="`
4023	`http`
1378	`://`
2503	`www`
13	`.`
13297	`google`
13	`.`
785	`com`
14	`/`
12947	`search`
30	`?`
80	`q`


This particular LLM uses a greedy/optimized tokenization method, almost always preferring the longest possible token, i.e. `://` will be preferred over `:` in full text (e.g. in training).

While URLs in training are encoded with token 1358 (`://`), our prompt makes the LLM see token `27` (`:`) instead, which throws off completion by artificially splitting `://`.
In fact, the model can be pretty sure that seeing token `27` (`:`) means what comes next is very unlikely to be anything that could have been encoded together with the colon using a "longer token" like `://`, since in the model's training data those characters would have been encoded together with the colon (an exception to this that we will discuss later is <a href="https://arxiv.org/abs/1804.10959">subword regularization</a> during training). The fact that seeing a token means both seeing the embedding of that token **and also** that whatever comes next wasn't compressed by the greedy tokenizer is easy to forget, but it is important in prompt boundaries.

Let's search over the string representation of all the tokens in the model's vocabulary, to see which ones start with a colon:

In [20]:
tokens = generator.tokenizer.convert_ids_to_tokens(range(generator.tokenizer.vocab_size))
colon_tokens = [i for i,t in enumerate(tokens) if t.startswith(":")]
print_tokens(colon_tokens)

len = 16
25	`:`
1378	`://`
3712	`::`
7479	`:\`
11097	`:"`
14079	`:/`
21912	`:-`
24022	`::::`
29164	`:{`
32105	`:'`
33250	`:[`
37498	`:(`
43661	`::::::::`
43922	`:#`
45299	`:,`
47715	`:]`


Note that there are **34** different tokens starting with a colon, and thus ending a prompt with a colon means the model will likely not generate completions with any of these 34 token strings. *This subtle and powerful bias can have all kinds of unintended consequences.* And this applies to **any** string that could be potentially extended to make a longer single token (not just `:`).  Even our "fixed" prompt ending with "http" has a built in bias as well, as it communicates to the model that what comes after "http" is likely not "s" (otherwise "http" would not have been encoded as a separate token):

In [21]:
http_tokens = [i for i,t in enumerate(tokens) if t.startswith("http")]
print_tokens(http_tokens)

len = 2
4023	`http`
5450	`https`


Lest you think this is an arcane problem that only touches URLs, remember that most tokenizers treat tokens differently depending on whether they start with a space, punctuation, quotes, etc, and thus **ending a prompt with any of these can lead to wrong token boundaries**, and break things:

In [22]:
# Accidentally adding a space, will lead to weird generation
raw_gen('I read a book about ')

'I read a book about ????????, which I never really made it to'

In [23]:
# No space, works as expected
raw_gen('I read a book about')

'I read a book about the great American hero.\n\n"He lived'

Another example of this is the "[" character. Consider the following prompt and completion:

In [24]:
raw_gen('An example ["like this"] and another example [')

'An example ["like this"] and another example [that has] a bunch of other little bugs that'

Why is the second string not quoted? Because by ending our prompt with the ' [' token, we are telling the model that it should not generate completions that match the following 27 longer tokens (one of which adds the quote character, `15640`):

In [25]:
space_bracket_tokens = [i for i,t in enumerate(tokens) if t .startswith("Ġ[")] # note the Ġ is converted to a space by the tokenizer
print_tokens(space_bracket_tokens)

len = 17
685	` [`
14631	` ["`
16410	` [[`
17635	` []`
23834	` […]`
25787	` [];`
25915	` [-`
26076	` [+`
26894	` [...]`
30138	` [*`
36338	` [*]`
37250	` ['`
46581	` [/`
47175	` [+]`
47527	` [(`
49074	` [|`
49082	` [&`


Token boundary bias happens everywhere. *About 70% of the 10k most common tokens for the model used above are prefixes of longer possible tokens, and so cause token boundary bias when they are the last token in a prompt.* Keeping track of all these possible extension biases during prompt design is impractical so most people just ignore them.

In [31]:
from tqdm import tqdm
# count the number of tokens that have longer extensions
count = 0
for i in tqdm(range(10000)):
    m = 0
    for j in range(generator.tokenizer.vocab_size):
        if tokens[j].startswith(tokens[i]):
            m += 1
        if m > 1:
            break
    if m > 1:
        count += 1
print(str(100*count/10000)+"%")

100%|██████████████████████████████| 10000/10000 [00:33<00:00, 299.73it/s]

68.47%


## Fixing unintended bias with "token healing"

What can we do to avoid these unintended biases? One option is to always end our prompts with tokens that cannot be extended into longer tokens (for example a role tag for chat-based models), but this is a severe limitation.  

Instead, `guidance` has a feature called "token healing", which automatically backs up the generation process by one token before the end of the prompt, then constrains the first token generated to have a prefix that matches the last token in the prompt. In our URL example, this would mean removing the `:`, and forcing generation of the first token to have a `:` prefix.   
Token healing allows users to express prompts however they wish, without worrying about token boundaries.

For example, let's re-run some of the URL examples above with token healing turned on (it's on by default for Transformer models, so we remove `token_healing=False`):

In [33]:
from guidance import models, gen

# load model from huggingface
lm = models.Transformers(model)

# With token healing we generate valid URLs, even when the prompt ends with a colon:
lm + 'The link is <a href="http:' + gen(max_tokens=10)

In [34]:
[str(lm + 'The link is <a href="http' + gen(max_tokens=10, temperature=1)) for i in range(10)]

['The link is <a href="https://www.facebook.com/mgb',
 'The link is <a href="http://www.facebook.com/power">facebook',
 'The link is <a href="http://30.timesofindia.com">',
 'The link is <a href="https://en.wikipedia.org/wiki/',
 'The link is <a href="https://twitter.com/KazzieBrow',
 'The link is <a href="http://reference.skyrim-dawnguard.',
 'The link is <a href="http://us911-blogs.blogspot.com/"',
 'The link is <a href="https://www.reddit.com/r/',
 'The link is <a href="https://t.co/COrPInst',
 'The link is <a href="http://forums.deploridad.org/']

Similarly, we don't have to worry about extra spaces:

In [35]:

# Accidentally adding a space will not impact generation
lm + 'I read a book about ' + gen(max_tokens=5)

In [36]:
# This will generate the same text as above 
lm + 'I read a book about' + gen(max_tokens=6)

Similarly, we now get quoted strings even when the prompt ends with a " [" token:

In [37]:
lm + 'An example ["like this"] and another example [' + gen(max_tokens=10)

## What about subword regularization?

If you are familiar with how language models are trained, you may be wondering how <a href="https://arxiv.org/abs/1804.10959">subword regularization</a> fits into all this. Subword regularization is a technique where during training sub-optimial tokenizations are randomly introduced to increase the model's robustness to token boundary issues. This means that the model does not always see the best tokenization. Subword regularization is great at helping the model be more robust to token boundaries, but it does not remove the bias that the model has towards the standard optimized (near greedy) tokenization. This means that while depending on the amount of subword regularization during training models may exhibit more or less token boundaries bias, all models still have this bias. And as shown above it can still have a powerful and unexpected impact on the model output.

## Conclusion

When you write prompts, remember that greedy tokenization can have a significant impact on how language models interpret your prompts, particularly when the prompt ends with a token that could be extended into a longer token. This easy-to-miss source of bias can impact your results in surprising and unintended ways.

To address to this, either end your prompt with a non-extendable token, or use something like `guidance`'s "token healing" feature so you can to express your prompts however you wish, without worrying about token boundary artifacts. 

## Appendix: Did we just get unlucky with the link example?

No, and random sampling can verify that:

In [38]:
# with the colon we almost always get an invalid link
[raw_gen('The link is <a href="http:', temp=1) for _ in range(5)]

['The link is <a href="http:".\n\nSource: https://www.drop',
 'The link is <a href="http:<br class="urn:preview:pre',
 'The link is <a href="http://www.sigco.com/com',
 'The link is <a href="http:">http://www.reddit.com/r',
 'The link is <a href="http: //www.museumofthumb.org']

In [39]:
# without the colon we always get a valid link
[raw_gen('The link is <a href="http', temp=1) for _ in range(5)]

['The link is <a href="http://www.huffingtonpost.co.',
 'The link is <a href="http://www.youtube.com/watch?v',
 'The link is <a href="http://t.co/tG9Tk',
 'The link is <a href="http://bit.ly/2fXfG',
 'The link is <a href="http://www.amazon.com/gp/trade']

<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>